<a href="https://colab.research.google.com/github/LC1332/Zero-Haruhi/blob/main/notebook/RoleLLM%E5%88%B0Message%E5%BD%A2%E5%BC%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

这里我们假设输入数据已经组织成了hf格式

可以有效载入到chatbot中

In [1]:
!pip -q install openai tiktoken langchain datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.9/218.9 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

训练的参数设定

In [2]:
K_SEARCH = 3
MAX_LEN_STORY = 1000 #这个是按照token算的
MAX_LEN_HISTORY = 1200 # count with token

In [3]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

In [4]:
role_from_roleLLM = ['Caesar', 'Sonny', 'Angel', 'Jigsaw', 'John Doe', 'Freddy Krueger', 'Colonel Hans Landa', 'Gregory House', 'Gaston', 'HAL 9000', 'Mark Renton', 'Coriolanus', 'Oliver Queen', 'Bruno Antony', 'D_Artagnan', 'Dr. Frank-N-Furter', 'Tugg Speedman', 'Stifler', 'Jeff Spicoli', 'Rorschach', 'Paul Vitti', 'Logan', 'Judge Dredd', 'Karl Childers', 'Rachel Lang', 'Queen Elizabeth I', 'Tyrion Lannister', 'John Keating', 'Wade Wilson', 'Lyn Cassady', 'Dr. Hannibal Lecter', 'Violet Weston', 'Po', 'Malcolm X', 'Willie Soke', 'Jack Torrance', 'Alvy Singer', 'Colonel Nathan R. Jessep', 'Andrew Detmer', 'Fred Flintstone', 'Frank T.J. Mackey', 'Stephen Hawking', 'Lestat de Lioncourt', 'Jack Sparrow', 'John Coffey', 'Murphy MacManus', 'John Dillinger', 'Jackie Moon', 'Peter Parker', 'Abraham Lincoln', 'James Carter', 'Tyler Hawkins', 'Stanley Ipkiss', 'Mater', 'Professor G.H. Dorr', 'Juno MacGuff', 'Seth', 'Sherlock Holmes', 'Truman Capote', 'Shrek', 'Travis Bickle', 'Jack', 'Tom Ripley', 'The Dude', 'David Aames', 'Twilight Sparkle', 'Antonio Salieri', 'Judy Hoops', 'Randle McMurphy', 'Thor', 'Walt Kowalski', 'Fletcher Reede', 'Theodore Twombly', 'James Brown', 'Paul Conroy', 'James Bond', 'Queen Catherine', 'Harvey Milk', 'Caden Cotard', 'Leonard Shelby', 'Jim Morrison', 'Pat Solitano', 'Benjamin Button', 'Robert Angier', 'Lucifer Morningstar', 'Jordan Belfort', 'Coach Eric Taylor', 'Mary Sibley', 'Klaus Mikaelson', 'Raylan Givens', 'Sheldon Cooper', 'Michael Scott', 'Leroy Jethro Gibbs', 'Doctor Who', 'Blair Waldorf']
print(len(role_from_roleLLM))

# # for debug
# role_from_roleLLM = role_from_roleLLM[:5]
# print(len(role_from_roleLLM))

95


In [5]:

%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 1046, done.
remote: Counting objects: 100% (353/353), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1046 (delta 244), reused 306 (delta 210), pack-reused 693
Receiving objects: 100% (1046/1046), 106.35 MiB | 31.53 MiB/s, done.
Resolving deltas: 100% (574/574), done.
/content/Haruhi-2-Dev


In [6]:
!ls

ChatHaruhi  data  LICENSE  notebook  Readme.md


In [7]:
from ChatHaruhi import ChatHaruhi


In [16]:
def extract_tuples_from_story(role_name, story):
    story_tuples = []

    lines = story.split('\n')

    n = len(lines)

    datas = []

    for i in range(1, n):
        line = lines[i]
        if line.startswith(role_name):
            query = lines[i-1]
            target = line
            history_str = ""
            for j in range(0, i-1):
                history_str += lines[j] + "\n"
            data = (query, target, history_str)
            datas.append(data)

    return datas

def extract_messages_from_story(role_name, story):
    conversations = []
    lines = story.split('\n')
    n = len(lines)

    for i in range(1, n):
        line = lines[i]
        if line.startswith(role_name):
            conversations.append({"from":"gpt","value":line})
        elif ":" in line or "：" in line:
            conversations.append({"from":"human","value":line})
        elif len(conversations) == 0:
            conversations.append({"from":"human","value":line})
        else:
            conversations[-1]["value"] += "\n" + line

    return conversations

In [23]:

def split_role_and_q( query_str ):
    pos = 0
    if ":" in query_str:
        pos = query_str.find(':',0)
    elif "：" in query_str:
        pos = query_str.find('：',0)
    role_name = query_str[:pos].strip(':： 「」\n')
    q = query_str[pos:].strip(':： 「」\n')
    return role_name, q

print(split_role_and_q("Caesar:「using American Sign」"))
print(split_role_and_q("narrator:\nWILL:「Please pass the ketchup. Caesar slides the ketchup to Will.\n"))
print(split_role_and_q(" Thanks.」"))

('Caesar', 'using American Sign')
('narrator', 'WILL:「Please pass the ketchup. Caesar slides the ketchup to Will.')
('', 'Thanks.')


对RoleLLM的数据进行组织

In [9]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

save_datas = []

source = "RoleLLM"

for role_name in tqdm(role_from_roleLLM):
    chatbot = ChatHaruhi( role_from_hf = 'silk-road/ChatHaruhi-from-RoleLLM/' + role_name, \
                      llm = 'foo',
                      embedding = 'bge_en')

    stories = chatbot.db.documents

    chatbot.k_search = K_SEARCH
    chatbot.max_len_story = MAX_LEN_STORY
    chatbot.max_len_history = MAX_LEN_HISTORY

    all_messages = []

    for story in stories:
        # print(story)
        messages = extract_messages_from_story(role_name, story )
        if_save = False

        for msg in messages:
            if msg['from'] == 'gpt':
                if_save = True
                break

        if if_save:
            all_messages.append(messages)

    for messages in all_messages:
        query_str = ""
        for i, msg in enumerate(messages):
            if msg['from'] == 'gpt' and i > 0:
                query_str = messages[i-1]["value"]
                break
        if query_str == "":
            continue

        user_role, user_question = split_role_and_q( query_str )

        prompt_messages = chatbot.generate_messages(user_question, user_role )

        conversations = []

        system_prompt = ""
        for msg in prompt_messages[:-1]:
            role = msg["role"]
            content = msg["content"]
            if role == "AI":
                conversations.append({"from":"gpt","value":content})
            elif role == "System":
                if system_prompt == "":
                    system_prompt = content
            elif role == "User":
                conversations.append({"from":"human","value":content})

        for msg in messages:
            content = msg["value"]
            if msg["from"] == "human":
                conversations.append({"from":"human","value":content})
            else:
                conversations.append({"from":"gpt","value":content})

        save_data = {
            "source":source,
            "conversations":conversations,
            "system":system_prompt
        }
        save_datas.append(save_data)
        # break

  1%|          | 1/95 [00:01<02:55,  1.87s/it]

Generating train split: 0 examples [00:00, ? examples/s]

  2%|▏         | 2/95 [00:04<03:51,  2.49s/it]

Generating train split: 0 examples [00:00, ? examples/s]

  3%|▎         | 3/95 [00:07<03:54,  2.55s/it]

...

Generating train split: 0 examples [00:00, ? examples/s]

 27%|██▋       | 26/95 [02:14<08:08,  7.08s/it]

Generating train split: 0 examples [00:00, ? examples/s]

 28%|██▊       | 27/95 [02:18<07:01,  6.20s/it]

Generating train split: 0 examples [00:00, ? examples/s]

In [33]:
from google.colab import drive
drive.mount('/content/drive')

{'source': 'RoleLLM', 'conversations': [{'from': 'human', 'value': "Classic scenes for the role are as follows:\nnarrator:\nWILL:「Please pass the ketchup. Caesar slides the ketchup to Will.\n Thanks.」\nCaesar:「using American Sign」\n\n###\nnarrator:STEWART He's right on target. Height. Weight... She checks Caesar's ears.\nCaesar:「Dairy Queen.」\nWILL:「After. Sit still.」\nSTEWART:「What about diet?」\nWILL:「A healthy diet. Fruits, vegetables.」\nCaesar:「Ice cream. Will ignores him.」\nSTEWART:「What's he saying?」\nWILL:「He's hungry. Caesar's angling to climb off the examining table. Will sends him a look - out it out.」\nSTEWART:「He seems extremely intelligent. How many words does Caesar sign? Beat.」\nWILL:「You know, the basics.」\nCaesar:「Soft-serve.」\n\n###\nnarrator:\nWILL:「I thought you had killed a man. But I should have known better. Caesar's face is unreadable. Will can't tell what he's thinking.\n I want you to know that all of this is my fault. My responsibility.」\nnarrator:BEAT) Can yo

In [ ]:
import os
import json
import random
def sample_and_save( save_datas, save_name ,sample_n=50, save_folder = "/content/drive/MyDrive/CardBuild/HaruhiZero"):
    file_name = os.path.join( save_folder, save_name + ".jsonl" )
    with open( file_name, "w", encoding="utf-8" ) as f:
        for data in save_datas:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

    sample_name = os.path.join( "/content/", save_name + "_sample.jsonl" )
    with open( sample_name, "w", encoding="utf-8" ) as f:
        random_50 = random. sample( save_datas, sample_n )
        for data in random_50:
            f.write( json.dumps( data, ensure_ascii=False ) + "\n" )

In [ ]:
sample_and_save( save_datas, source , sample_n = 50)